`--- Day 9: Marble Mania ---`

In [1]:
import unittest
import re

You talk to the Elves while you wait for your navigation system to initialize. To pass the time, they introduce you to their favorite marble game.

The Elves play this game by taking turns arranging the marbles in a circle according to very particular rules. The marbles are numbered starting with 0 and increasing by 1 until every marble has a number.

First, the marble numbered 0 is placed in the circle. At this point, while it contains only a single marble, it is still a circle: the marble is both clockwise from itself and counter-clockwise from itself. This marble is designated the current marble.

Then, each Elf takes a turn placing the lowest-numbered remaining marble into the circle between the marbles that are 1 and 2 marbles clockwise of the current marble. (When the circle is large enough, this means that there is one marble between the marble that was just placed and the current marble.) The marble that was just placed then becomes the current marble.

However, if the marble that is about to be placed has a number which is a multiple of 23, something entirely different happens. First, the current player keeps the marble they would have placed, adding it to their score. In addition, the marble 7 marbles counter-clockwise from the current marble is removed from the circle and also added to the current player's score. The marble located immediately clockwise of the marble that was removed becomes the new current marble.

For example, suppose there are 9 players. After the marble with value 0 is placed in the middle, each player (shown in square brackets) takes a turn. The result of each of those turns would produce circles of marbles like this, where clockwise is to the right and the resulting current marble is in parentheses:
```
[-] (0)
[1]  0 (1)
[2]  0 (2) 1 
[3]  0  2  1 (3)
[4]  0 (4) 2  1  3 
[5]  0  4  2 (5) 1  3 
[6]  0  4  2  5  1 (6) 3 
[7]  0  4  2  5  1  6  3 (7)
[8]  0 (8) 4  2  5  1  6  3  7 
[9]  0  8  4 (9) 2  5  1  6  3  7 
[1]  0  8  4  9  2(10) 5  1  6  3  7 
[2]  0  8  4  9  2 10  5(11) 1  6  3  7 
[3]  0  8  4  9  2 10  5 11  1(12) 6  3  7 
[4]  0  8  4  9  2 10  5 11  1 12  6(13) 3  7 
[5]  0  8  4  9  2 10  5 11  1 12  6 13  3(14) 7 
[6]  0  8  4  9  2 10  5 11  1 12  6 13  3 14  7(15)
[7]  0(16) 8  4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[8]  0 16  8(17) 4  9  2 10  5 11  1 12  6 13  3 14  7 15 
[9]  0 16  8 17  4(18) 9  2 10  5 11  1 12  6 13  3 14  7 15 
[1]  0 16  8 17  4 18  9(19) 2 10  5 11  1 12  6 13  3 14  7 15 
[2]  0 16  8 17  4 18  9 19  2(20)10  5 11  1 12  6 13  3 14  7 15 
[3]  0 16  8 17  4 18  9 19  2 20 10(21) 5 11  1 12  6 13  3 14  7 15 
[4]  0 16  8 17  4 18  9 19  2 20 10 21  5(22)11  1 12  6 13  3 14  7 15 
[5]  0 16  8 17  4 18(19) 2 20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[6]  0 16  8 17  4 18 19  2(24)20 10 21  5 22 11  1 12  6 13  3 14  7 15 
[7]  0 16  8 17  4 18 19  2 24 20(25)10 21  5 22 11  1 12  6 13  3 14  7 15
```
The goal is to be the player with the highest score after the last marble is used up. Assuming the example above ends after the marble numbered 25, the winning score is 23+9=32 (because player 5 kept marble 23 and removed marble 9, while no other player got any points in this very short example game).

Here are a few more examples:
```
    10 players; last marble is worth 1618 points: high score is 8317
    13 players; last marble is worth 7999 points: high score is 146373
    17 players; last marble is worth 1104 points: high score is 2764
    21 players; last marble is worth 6111 points: high score is 54718
    30 players; last marble is worth 5807 points: high score is 37305
```
**What is the winning Elf's score?**


In [2]:
players, max_marble = map(int, re.findall(r'-?\d+', open('input.txt').read()))

In [3]:
test_values = [(9, 25, 32),
               (10, 1618, 8317),
               (13, 7999, 146373),
               (17, 1104, 2764),
               (21, 6111, 54718),
               (30, 5807, 37305)]

In [4]:
class TestPart1(unittest.TestCase):
    def test_marblescore(self):
        for players, max_marble, score in test_values:
            with self.subTest(players=players):
                self.assertEqual(marbleScore(players, max_marble), score)

In [5]:
class Marble(object):
    
    def __init__(self, number):
        self.number = number
        self._prev = self
        self._next = self
        
    def nextMarble(self):
        return self._next
    
    def prevMarble(self):
        return self._prev
    
    def dump(self):
        l = []
        l.append(self.number)
        x = self.nextMarble()
        while x != self:
            l.append(x.number)
            x = x.nextMarble()
        print(' '.join(str(n) for n in l))
        
    def __str__(self):
        return f'Marble({self.number}), prev={self._prev.number}, next={self._next.number}'
    
    __repr__ = __str__
        

In [6]:
def marbleScore(players, max_marble):
    scores = [0] * players
    current = Marble(0)
    current._next = current
    current._prev = current
    
    for marble in range(1, max_marble + 1):
        if marble % 23 == 0:
            player = marble % players
            scores[player] += marble
            seven_before = current.prevMarble().prevMarble().prevMarble().prevMarble().prevMarble().prevMarble().prevMarble()
            scores[player] += seven_before.number
            seven_before.prevMarble()._next = seven_before.nextMarble()
            seven_before.nextMarble()._prev = seven_before.prevMarble()
            current = seven_before.nextMarble()
            del seven_before
        else:
            new = Marble(marble)
            one_clockwise = current.nextMarble()
            two_clockwise = current.nextMarble().nextMarble()
            new._prev = one_clockwise
            new._next = two_clockwise
            one_clockwise._next = new
            two_clockwise._prev = new
            current = new
            
    return max(scores)

# Run the tests

In [7]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestPart1)
unittest.TextTestRunner(verbosity=2).run(suite)

test_marblescore (__main__.TestPart1) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.047s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# Part 1

In [8]:
%%timeit -n 1 -r 1
marbleScore(players, max_marble)

120 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Part 2

In [9]:
%%timeit -n 1 -r 1
marbleScore(players, max_marble * 100)

13.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
